# **Fine-Tunning Gemma For Code Generation Using Unsloth**

**Install Libraries**

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

**Import Libraries**

In [2]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


**Define Parameters**

In [3]:
# Set the maximum sequence length to 8192 tokens.
max_seq_length = 8192

# Set the data type to None for automatic detection.
dtype = None

# Set the load_in_4bit flag to True to load the model weights in 4-bit precision.
load_in_4bit = True

**Download Gemma Pre-Trained Model**

In [4]:
# Load the pre-trained model from the 'unsloth/gemma-7b-bnb-4bit' repository.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-7b-bnb-4bit",

    # Set the maximum sequence length to the value defined earlier.
    max_seq_length = max_seq_length,

    # Set the data type to the value defined earlier.
    dtype = dtype,

    # Set the load_in_4bit flag to the value defined earlier.
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Gemma patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

**Create LoRA Adapters For The Fine-Tunned Model**

In [5]:
# Create a PEFT model with the given parameters
model = FastLanguageModel.get_peft_model(
    model,
    r=16, # LoRa Rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True
)

Unsloth 2024.8 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


LoRA adapters for the Gemma 7B model will alow us to fine-tune the model on different types of dataset.

**Define Function To Convert Training Dataset Into Text Format**

In [6]:
def formatted_train(x):

  if x['input']:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {x['instruction']}

    ### Input:
    {x['input']}

    ### Response:
    {x['output']}<eos>"""

  else:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {x['instruction']}

    ### Response:
    {x['output']}<eos>"""

  return formatted_text

**Load and Process Dataset**

In [7]:
def prepare_train_data(data_id):
    data = load_dataset(data_id, split="train")
    data_df = data.to_pandas()
    data_df["formatted_text"] = data_df[["input", "output",
    "instruction"]].apply(formatted_train, axis=1)
    data = Dataset.from_pandas(data_df)
    return data

data_id = "TokenBender/code_instructions_122k_alpaca_style"

data = prepare_train_data(data_id)

Generating train split:   0%|          | 0/121959 [00:00<?, ? examples/s]

**Fine-Tune Gemma For Code Dataset**

In [9]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = data,
    dataset_text_field = "formatted_text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 10,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/121959 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


**Define Trainer**

In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 121,959 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 50,003,968


Step,Training Loss
1,1.308700
2,1.268900
3,1.343000
4,1.219000
5,0.938900
6,0.902900
7,0.622500
8,0.695300
9,0.608800
10,0.665100


**Define Helper Function**

In [11]:
def format_test(x):

  if x['input']:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {x['instruction']}

    ### Input:
    {x['input']}

    ### Response:
    """

  else:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {x['instruction']}

    ### Response:
    """

  return formatted_text

**Display Prompt Example**

In [12]:
Prompt = format_test(data[155])
print(Prompt)

Below is an instruction that describes a task.     Write a response that appropriately completes the request.

    ### Instruction:
    Create a way to encrypt a message using a key in Python.

    ### Input:
    message = "Hello world!"
key = "secret"

    ### Response:
    


**Run Fine-Tunned Model To Generate Code For Given Prompt**

In [13]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    Prompt
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<bos>Below is an instruction that describes a task.     Write a response that appropriately completes the request.

    ### Instruction:
    Create a way to encrypt a message using a key in Python.

    ### Input:
    message = "Hello world!"
key = "secret"

    ### Response:
    import hashlib

def encrypt(message, key):
    hash = hashlib.md5(key.encode()).hexdigest()
    return hash + message

encrypted_message = encrypt(message, key)
print(encrypted_message)<eos>
